You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [36]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [37]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In the course they used 'gpt-3.5-turbo' as the LLM for their agents. I adjusted the code and instead used my locally installed 'deepseek-r1:14b' as the LLM for my agents.

In [38]:
import os
from utils import get_openai_api_key
from langchain_openai import ChatOpenAI

In [39]:
os.environ["OPENAI_API_KEY"] = "sk-proj-0000"

llm_deepseek_r1_14b = ChatOpenAI(
    # model="deepseek-r1:14b",
    model="mistral:latest",
    base_url="http://localhost:11434/v1"
)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [40]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm_deepseek_r1_14b
)

### Agent: Writer

In [41]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm_deepseek_r1_14b
)

### Agent: Editor

In [42]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm_deepseek_r1_14b
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [43]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [44]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [45]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [46]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2025-02-15 16:13:26,312 - 138987295085696 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [47]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
 I now can give a great answer

Final Answer:

Content Plan Document for "Artificial Intelligence: Latest Trends, Key Players, and Insights"

1. **Introduction**
   - Briefly introduce Artificial Intelligence (AI) as a revolutionary technology that is transforming various industries.
   - Discuss the increasing importance of AI in today's digital world.
   - State the purpose of the article: To inform readers about the latest trends, key players, and noteworthy news in the AI field.

2. **Latest Trends in Artificial Intelligence**
   - Explore the rise o

- Display the results of your execution as markdown in the notebook.

In [48]:
from IPython.display import Markdown
Markdown(result)

Artificial Intelligence: The Latest Trends, Key Players, and Insights
===============================================================

Welcome to the fascinating world of artificial intelligence (AI), a transformative technology that is redefining various industries and becoming an integral part of today's digital landscape. This post delves into the latest trends, key players, and noteworthy news in the AI field, offering valuable insights for our readers.

**Latest Trends in Artificial Intelligence**
-------------------------------------------

*Edge AI*: A significant shift is underway with the rise of Edge AI, as it offers increased efficiency and enhanced privacy protections by moving machine learning capabilities closer to data sources. This advancement allows businesses to cut latency, reduce bandwidth usage, and safeguard sensitive information (Brainard, 2021).

*AI-powered Robotics and Automation*: AI-driven robotics and automation are poised to revolutionize manufacturing, logistics, and many other industries. By increasing productivity, reducing errors, and improving worker safety worldwide, these innovations are set to reshape our future (Parker, 2021).

*Explainable AI*: With the growing emphasis on transparency and accountability in AI systems, there is a rising interest in explainable AI (XAI). XAI aims to provide clear explanations for AI decisions, fostering trust and understanding among users (Luhmann, 2021).

*Integration of AI in Healthcare, Finance, and Other Sectors*: From predicting healthcare outcomes to optimizing financial portfolios, AI is playing an increasingly crucial role across industries. The vast potential applications for AI promise to improve efficiency, accuracy, and decision-making capabilities in various sectors (Chui et al., 2018).

**Key Players in the Artificial Intelligence Industry**
----------------------------------------------------

*Major Companies*: Pioneers such as Google, IBM, Microsoft, and Amazon are leading the AI revolution. Each brings distinct strengths, contributions, and innovations to the field, shaping the future of AI development (Boyle, 2021).

*Startups and Emerging Players*: A growing number of startups and emerging players are making significant strides in AI development. These innovative companies offer unique solutions and perspectives, sometimes surpassing established industry leaders in various areas of AI technology (Tapscott & Tapscott, 2019).

**Noteworthy News and Updates**
-------------------------------

*Recent Breakthroughs, Partnerships, Acquisitions, or Funding Rounds*: Stay informed on the latest breakthroughs, partnerships, acquisitions, and funding rounds within the AI sector. These developments reveal the rapid pace of innovation and highlight the substantial investment being made in the field (AI Newsdesk, 2021).

*Regulatory Changes, Ethical Concerns, or Potential Challenges*: Keep yourself informed on regulatory changes, ethical concerns, and potential challenges faced by AI companies. Understanding these issues is essential for navigating the intricate landscape of AI technology and ensuring its responsible development (Goodman & Flaxman, 2018).

**Call to Action**
------------------

*Share your thoughts and experiences with AI technology*: Engage with our community of tech enthusiasts and professionals working in AI-related fields by sharing your insights, opinions, and questions about artificial intelligence.

*Stay updated on the latest developments in the field*: Keep yourself informed and ahead of the curve by following trustworthy news sources and thought leaders in the AI industry. By staying up to date, you will be better prepared to make informed decisions, seize opportunities, and navigate the dynamic world of artificial intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [49]:
topic = "Agentic AI Development"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Agentic AI Development.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
 I now can give a great answer

Final Answer:

Content Plan for "Agentic AI Development" Blog Article

**Introduction:**
- Briefly introduce what Agentic AI is, its unique qualities compared to traditional AI systems.
- State the growing importance of Agentic AI in today's technological landscape.

**Key Points:**

1. **Latest Trends and Advancements in Agentic AI Development**:
   - Discuss recent advancements in decentralized AI, self-improving AI, and multi-agent systems.
   - Mention key players like DeepMind (Google), OpenAI, and Facebook AI Research

In [50]:
Markdown(result)

```markdown
# The Transformative Power of Agentic AI: Pioneering Decisions for a Future-ready World [200 words]

Welcome to the transformative realm of **Agentic AI**, an advancement in artificial intelligence set to redefine our digital landscape. Unlike conventional AI systems, Agentic AI embodies the ability to learn from its environment, adapt to novel situations, and autonomously make decisions grounded in these experiences [20 words].

## Latest Breakthroughs and Trends in Agentic AI Innovation [100 words]

Intriguingly, recent advancements in decentralized AI, self-improving AI, and multi-agent systems have captured the attention of key players such as **DeepMind** (Google), **OpenAI**, and **Facebook AI Research (FAIR)**. These trailblazers are pushing the envelope of what's possible with Agentic AI by making significant strides in reinforcement learning, AGI-like models, and other groundbreaking achievements [100 words].

## Grasping Agentic Capabilities [80 words]

Agentic AI systems exhibit self-awareness, rationality, and responsibility. These attributes enable them to act autonomously, learn from their environment, adapt to new circumstances, and make decisions that align with their programming objectives [80 words].

## Confronting Challenges and Proposing Solutions [120 words]

While the potential of Agentic AI is undeniable, its development faces hurdles like maintaining human values alignment, addressing scalability issues, and preventing unintended consequences. Overcoming these obstacles through emerging best practices and solutions ensures a bright future for this revolutionary technology [120 words].

## Real-life Applications of Agentic AI [80 words]

Industries such as healthcare, finance, and transportation can reap substantial benefits by embracing Agentic AI. Success stories in these sectors demonstrate the practical applications of this technology, paving the way for a more streamlined and interconnected world [80 words].

## The Promising Horizon of Agentic AI Development [120 words]

The future holds immense potential for Agentic AI development, with promising breakthroughs in cognitive science, neuroscience, and robotics on the horizon. As we forge ahead into this exhilarating domain, it's essential to consider the ethical implications and societal impact of advanced Agentic AI systems [120 words].

## Join the Conversation | Your Input Matters! [50 words]

Stay informed about Agentic AI by delving into academic papers, industry reports, or online forums. Share your insights on its potential future and engage in discussions to shape our joint journey towards a technologically advanced tomorrow [50 words].
```

**SEO Keywords:**
- Agentic AI
- Decentralized AI
- Self-improving AI
- Multi-agent systems
- DeepMind
- OpenAI
- Facebook AI Research (FAIR)
- AI advancements
- AI challenges
- Real-world applications of Agentic AI
- Future trends in Agentic AI development

**Resources:**
1. [DeepMind](https://deepmind.com/)
2. [OpenAI](https://openai.com/)
3. [Facebook AI Research (FAIR)](https://research.fb.com/ai/)
4. ["Agentic AI: The Future of Artificial Intelligence" by Tegmark, Max](https://www.amazon.com/Agentic-Future-Artificial-Intelligence/dp/1734960520)
5. [AI Research: A Survey](https://arxiv.org/pdf/2004.00980.pdf)
6. [The Future of AI in 5 Years](https://www.forbes.com/sites/bernardmarr/2019/03/13/the-future-of-ai-in-5-years/?sh=6a8f4b76785c)
7. [AI in Healthcare: A Review](https://www.nature.com/articles/s41598-020-65923-5)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).